In [4]:
from xgboost import XGBClassifier 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

/opt/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
import pandas as pd
df = pd.read_csv('./All.csv')
df = df.sample(frac=1)
np.unique(df['URL_Type_obf_Type'])

array(['Defacement', 'benign', 'malware', 'phishing', 'spam'],
      dtype=object)

In [6]:
df['URL_Type_obf_Type'] = pd.Categorical(df['URL_Type_obf_Type'])
df['URL_Type_obf_Type'] = df['URL_Type_obf_Type'].cat.codes
X = df.drop(['URL_Type_obf_Type'], axis=1)
y = df['URL_Type_obf_Type']

In [7]:
X.shape

(36707, 79)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)

In [ ]:
weights = pd.Series(y_train).value_counts().values / y_train.size
all_weights = np.zeros(y_train.size)
for label in np.unique(y_train):
    all_weights[y_train == label] = weights[label]
xgb = XGBClassifier(n_estimators=5000, learning_rate=0.05, gamma=0, subsample=0.7,
                            objective="multi:softprob",
                            num_class=len(np.unique(y_train)),
                            eval_metric="mlogloss", random_state=93, use_label_encoder=False)
xgb = xgb.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)],
                        early_stopping_rounds=10,
                        sample_weight=all_weights)

[11:13:28] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-mlogloss:1.52425	validation_1-mlogloss:1.52489
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 10 rounds.
[1]	validation_0-mlogloss:1.45092	validation_1-mlogloss:1.45222
[2]	validation_0-mlogloss:1.38367	validation_1-mlogloss:1.38539
[3]	validation_0-mlogloss:1.32165	validation_1-mlogloss:1.32401
[4]	validation_0-mlogloss:1.26374	validation_1-mlogloss:1.26692
[5]	validation_0-mlogloss:1.21208	validation_1-mlogloss:1.21580
[6]	validation_0-mlogloss:1.16401	validation_1-mlogloss:1.16853
[7]	validation_0-m

In [ ]:
# predict classification report
y_pred = xgb.predict(X_test)
# print classification report
print(classification_report(y_test, y_pred))

In [ ]:
import pickle
with open("xgb_good", "wb") as file:
    pickle.dump(xgb, file)

In [ ]:
# strategy = tf.distribute.MirroredStrategy()
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Dropout
model = Sequential()
model.add(Dense(40, activation="relu",input_shape=(79,1)))
model.add(Dropout(0.3))
model.add(Dense(20, activation="relu"))
model.add(Dense(10, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),metrics=["accuracy",tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])
model.summary()

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20)
history = model.fit(X_train,y_train,batch_size=200,epochs=500,
                    validation_split=0.05,class_weight=weights,
                    use_multiprocessing=True,workers=8,callbacks=[callback])

In [ ]:
history